## Main file for project

### Data preprocessing 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image
import torch
from torch import nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Munch data preprocessing, only need to run once
########################################
# make sure the data are downloaded from 
# https://www.kaggle.com/code/isaienkov/how-to-start-with-munch-s-paintings
# and set fileDir to the folder
########################################
# Monet painting data are from 
# https://www.kaggle.com/code/dimitreoliveira/introduction-to-cyclegan-monet-paintings/notebook
# already cleaned and reshaped

# This is the directory of the unzipped file you download
fileDir = "../../../Desktop/NU/Deep_Learning/archive/"
imgIndex = pd.read_csv(fileDir + "/edvard_munch.csv")
fileList = imgIndex["filename"]

# load each individual image, resize and save
processDir = fileDir + "/munch_processed"
if not os.path.exists(processDir):
    os.makedirs(processDir)


In [3]:
# Prepare Pytorch dataset

class MyDataset(torch.utils.data.Dataset): 
    def __init__(self, MunchDir, MonetDir, MunchName, MonetName, transform=None, target_transform=None): 
        imgs = []                      
        for name in MunchName:
            imgs.append((name,1))
        for name in MonetName:
            imgs.append((name,0))
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
 
    def __getitem__(self, index):  
        fn, label = self.imgs[index]
        if label == 1:
            img = Image.open(MunchDir + '/' + fn).convert('RGB')
        if label == 0:
            img = Image.open(MonetDir + '/' + fn).convert('RGB')
        if self.transform is not None:
            img = self.transform(img) 
        return img, label
 
    def __len__(self): 
        return len(self.imgs)


In [8]:
# training and testing preperation

# Munch processed image
MunchDir = '../../../Desktop/NU/Deep_Learning/archive/munch_processed'
MunchFileName = np.array(os.listdir(MunchDir))

# Monet processed image
MonetDir = '../../../Desktop/NU/Deep_Learning/archive/munch_processed'
MonetFileName = np.array(os.listdir(MonetDir))

# training set size
np.random.seed(100)
trainRatio = 0.7

trainMunchIdx = np.random.choice(len(MunchFileName), int(trainRatio * len(MunchFileName)), replace=False)
trainMonetIdx = np.random.choice(len(MonetFileName), int(trainRatio * len(MonetFileName)), replace=False)

trainMunch = MunchFileName[trainMunchIdx]
trainMonet = MonetFileName[trainMonetIdx]

testMunch = np.delete(MunchFileName, trainMunchIdx)
testMonet = np.delete(MonetFileName, trainMonetIdx)


In [28]:
# ResNet Model
# Refer to the model Provided by Kaiming, H., Xiangyu, Z. et al. Deep Residual Learning for Image Recognition(2015),https://arxiv.org/abs/1512.03385 

# Define basic ResNet Blocks: ConvBlock and IdentityBlock
class ConvBlock(nn.Module):
    '''
    The basic element of ResNet: Convolutional Block, including 3 convolution layers
    '''
    def __init__(self, in_channel:int, filters:list, strides = 1):
        '''
        :param in_channel: input dim
        :param filters: The number of filters for each concolutional layer (lenghth = 3).
        :param strides: the stride for the first layer, default 1.
        '''
        super(ConvBlock,self).__init__()
        F1, F2, F3 = filters
        self.stage = nn.Sequential(
            nn.Conv2d(in_channel,F1,1,stride=strides, padding=0, bias=False),
            nn.BatchNorm2d(F1),
            nn.ReLU(True),
            nn.Conv2d(F1,F2,3,stride=1, padding=1, bias=False),
            nn.BatchNorm2d(F2),
            nn.ReLU(True),
            nn.Conv2d(F2,F3,1,stride=1, padding=0, bias=False),
            nn.BatchNorm2d(F3),
        )
        self.shortcut_1 = nn.Conv2d(in_channel, F3, 1,stride = strides, padding=0, bias=False)
        self.batch_1 = nn.BatchNorm2d(F3)
        self.relu_1 = nn.ReLU(True)
        
    def forward(self, X):
        X_shortcut = self.shortcut_1(X)
        X_shortcut = self.batch_1(X_shortcut)
        X = self.stage(X)
        X = X + X_shortcut
        X = self.relu_1(X)
        return X    
    
class IndentityBlock(nn.Module):
    '''
    Simliar to ConvBlock, but works with ConvBlock as Bottle Neck.
    '''
    def __init__(self, in_channel:int, filters: list):
        '''
        :param in_channel: input dim
        :param filters: The number of filters for each concolutional layer (lenghth = 3).
        '''
        super(IndentityBlock,self).__init__()
        F1, F2, F3 = filters
        self.stage = nn.Sequential(
            nn.Conv2d(in_channel,F1,1,stride=1, bias=False),
            nn.BatchNorm2d(F1),
            nn.ReLU(True),
            nn.Conv2d(F1,F2,3,stride=1, padding=1, bias=False),
            nn.BatchNorm2d(F2),
            nn.ReLU(True),
            nn.Conv2d(F2,F3,1,stride=1, padding=0, bias=False),
            nn.BatchNorm2d(F3),
        )
        self.relu_1 = nn.ReLU(True)
        
    def forward(self, X):
        X_shortcut = X
        X = self.stage(X)
        X = X + X_shortcut
        X = self.relu_1(X)
        return X
    
class ResNet(nn.Module):
    '''
    ResNet
    '''
    def __init__(self, n_class: int):
        '''
        :param n_class: output dimension.
        '''
        super(ResNet,self).__init__()
        self.stage1 = nn.Sequential(
            nn.Conv2d(3,64,7,stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(3,2,padding=1),
        )
        self.stage2 = nn.Sequential(
            ConvBlock(64, filters=[64, 64, 256], strides=1),
            IndentityBlock(256, [64, 64, 256]),
            IndentityBlock(256, [64, 64, 256]),
        )
        self.stage3 = nn.Sequential(
            ConvBlock(256, filters=[128, 128, 512], strides=2),
            IndentityBlock(512, [128, 128, 512]),
            IndentityBlock(512, [128, 128, 512]),
            IndentityBlock(512, [128, 128, 512]),
        )
        self.stage4 = nn.Sequential(
            ConvBlock(512, filters=[256, 256, 1024], strides=2),
            IndentityBlock(1024, [256, 256, 1024]),
            IndentityBlock(1024, [256, 256, 1024]),
            IndentityBlock(1024, [256, 256, 1024]),
            IndentityBlock(1024, [256, 256, 1024]),
            IndentityBlock(1024, [256, 256, 1024]),
        )
        self.stage5 = nn.Sequential(
            ConvBlock(1024, filters=[512, 512, 2048], strides=2),
            IndentityBlock(2048, [512, 512, 2048]),
            IndentityBlock(2048, [512, 512, 2048]),
        )
        self.pool = nn.AvgPool2d(2,2,padding=1)
        self.fc = nn.Sequential(
            nn.Linear(2048*5*5,n_class)
        )
    
    def forward(self, X):
        out = self.stage1(X)
        out = self.stage2(out)
        out = self.stage3(out)
        out = self.stage4(out)
        out = self.stage5(out)
        out = self.pool(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 256
batchsz = 64
lr = 0.0002
epoches = 80

model = ResNet(2)
optim = torch.optim.Adam(model.parameters(), lr=lr)
BCE_loss = torch.nn.CrossEntropyLoss()
# Load data

train_data = MyDataset(MunchDir, MonetDir, trainMunch, trainMonet, transform=transforms.ToTensor())
test_data = MyDataset(MunchDir, MonetDir, testMunch, testMonet, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_data, batch_size=batchsz, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batchsz, shuffle = False)

for epoch in range(epoches):
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    for batch_idx, batch_data in enumerate(train_loader,1):
        x, y = batch_data
        x = x.float().to(device)
        ly = y.long().to(device)

        out = model(x)
        loss = BCE_loss(out, y)
        running_loss += loss.item() * y.size(0)
        _,pred = torch.max(out,1)  
        num_correct = (pred == y).sum()
        running_acc += num_correct.item()
        
        optim.zero_grad()
        loss.backward() 
        optim.step() 

    if (epoch+1) % 10 == 0:
        print('Train{} epoch, Loss: {:.6f},Acc: {:.6f}'.format(epoch+1,running_loss / (len(train_data)),
                                                               running_acc / (len(train_data))))
        model.eval()
        eval_loss = 0
        eval_acc = 0
        for data in test_loader:
            x,y = data
            x = x.float().to(device)
            y = y.long().to(device)
            out = model(x)
            loss = BCE_loss(out, y)
            eval_loss += loss.item() * y.size(0)
            _,pred = torch.max(out,1)   
            num_correct = (pred == y).sum() 
            eval_acc += num_correct.item() 

        print('Test Loss:{:.6f},Acc: {:.6f}'
            .format(eval_loss/ (len(test_data)),eval_acc * 1.0/(len(test_data))))


        



torch.Size([64, 2048, 5, 5])
torch.Size([64, 2048, 5, 5])


KeyboardInterrupt: 